# Analyse des réponses PIX

On commence par récupérer toutes les réponses depuis la base.

In [34]:
import os

# ID de l'épreuve
challenge_id = "rec6PW03oyqtPZ9rK" # pomme & arbre
# rectN26toxkJmt9S4 : perrières
# rec6PW03oyqtPZ9rK : pomme & arbre

query = r"SELECT * from answers where \"challengeId\" = '%s'" % challenge_id
os.system('echo "COPY (%s) TO STDOUT with CSV HEADER" | psql pg_production -o "pomme-arbre.csv"' % query)

0

On slugifie chaque réponse et on compte les plus fréquentes.

In [35]:
from collections import Counter
from slugify import slugify

samples = Counter()

nb_answers = 0
nb_pass = 0
with open('pomme-arbre.csv') as f:
    next(f)  # On saute la première ligne (noms des colonnes)
    for line in f:
        data = line.split(',')
        value = data[1]  # La réponse se trouve en 2e
        if value != '#ABAND#':
            samples[slugify(value)] += 1
        else:
            nb_pass += 1
        nb_answers += 1

print(nb_pass, 'abandons')
print(len(samples.keys()), 'différentes réponses parmi les', nb_answers, 'réponses\n')
print('Les plus fréquentes :')
for k, v in samples.most_common(10):
    print('%s (%d occurrences)' % (k, v))

3143 abandons
1146 différentes réponses parmi les 10778 réponses

Les plus fréquentes :
la-pomme-ne-tombe-jamais-loin-de-l-arbre (3581 occurrences)
la-pomme-ne-tombe-jamais-loin-de-larbre (552 occurrences)
la-pomme-ne-tombe-pas-loin-de-l-arbre (375 occurrences)
proverbe-la-pomme-ne-tombe-jamais-loin-de-l-arbre (341 occurrences)
on-peut-compter-le-nombre-de-pommes-dans-un-arbre (233 occurrences)
proverbe-pomme-arbre (106 occurrences)
pomme-arbre (93 occurrences)
la-pomme-tombe-de-l-arbre (78 occurrences)
la-pomme-ne-tombe-jamais-tres-loin-de-l-arbre (61 occurrences)
pommier (55 occurrences)


In [37]:
from leven import levenshtein
import numpy as np
from sklearn.cluster import dbscan

data = list(samples.keys())

def lev_metric(x, y):
    i, j = int(x[0]), int(y[0])     # extract indices
    return levenshtein(data[i], data[j])

X = np.arange(len(data)).reshape(-1, 1)
groups = dbscan(X, metric=lev_metric, eps=3, min_samples=5)
core_samples, labels = groups

bundles = {}
for i, sample in enumerate(data):
    bundles.setdefault(labels[i], []).append(sample)
for key in bundles:
    print('# Type', key)
    print(len(bundles[key]), 'réponses dont :')
    for answer in bundles[key][:15]:
        print('-', answer)
    print()

# Type 0
93 réponses dont :
- 
- cc
- ff
- pommes
- dapi
- pommier
- hnfr
- f-f
- pompom
- fruit
- dfsdf
- tg
- deded
- arbrer
- aple

# Type 1
5 réponses dont :
- on-peut-compter-le-nombre-de-pommes-dans-un-arbre-mais-on-ne-peut-pas-compter-le-nombre-d-arbre-dans-une-pomme
- on-peut-compter-le-nombre-de-pommes-dans-un-arbre-mais-on-ne-peut-pas-compter-le-nombre-d-arbres-dans-un-pomme
- on-peut-compter-le-nombre-de-pomme-dans-un-arbre-mais-on-ne-peut-pas-compter-le-nombre-d-arbre-dans-une-pomme
- on-peut-compter-le-nombre-de-pommes-dans-un-arbre-mais-l-on-ne-peut-pas-compter-le-nombre-d-arbres-dans-une-pomme
- on-peut-compter-le-nombre-de-pommes-dans-un-abre-mais-on-ne-peut-pas-compter-le-bnombre-d-arbres-dans-une-pomme

# Type 2
132 réponses dont :
- la-pomme-ne-tombre-jamais-loin-de-son-arbre
- la-pomme-ne-tombe-jamais-loin-de-l
- les-pommes-ne-tombe-jamais-loin-de-l-arbre
- une-pomme-ne-tombe-jamais-loin-de-l-arbre
- la-pomme-ne-tombe-jamais-moins-de-l-arbre
- la-mme-ne-tombe-jamais

Pour info, pour la rue des Perrières j'ai obtenu :

# Type 0 (FAUX)
56 réponses dont :
- 
- 38
- c
- laval
- a
- e
- montcul
- 633
- mon
- et
- ggg
- 1
- mont
- rue
- rue-rue

# Type 1 (~  rue de la Couteauderie, VRAI)
104 réponses dont :
- rue-de-la-courtauderiez
- la-coutaudrie
- rue-de-la-couteuderie
- rue-de-la-courtauderie
- couteaudrie
- rue-de-la-cuteauderie
- la-couteaudene
- de-la-coutauderie
- cautederie
- la-couteaudrerie
- rue-couteauderie
- rue-de-ma-couteauderie
- rue-de-couteuderie
- la-coutedrie
- la-couteaderie

# Type 2 (~ rue des Perrières, FAUX)
7 réponses dont :
- rue-des-perriers
- rue-de-perrier
- rue-de-perries
- rue-de-perrieres
- rue-des-peupliers
- rue-des-arrieres
- rue-des-perrieres

# Type 3 (FAUX)
10 réponses dont :
- numa-3
- num1-1
- num1-3
- numd-1
- numa-2
- num1-4
- num1-2
- num1-8
- num4-1
- numb-1

# Type 4 (VRAI)
7 réponses dont :
- proverbe-la-pomme-ne-tombe-jamais-loin-de-son-arbre
- proverbe-une-pomme-ne-tombe-jamais-loin-de-l-arbre
- proverbe-la-pomme-ne-tombe-jamais-loin-de-l-arbre
- proverbe-la-pomme-ne-tombe-jamais-loin-de-larbre
- proverbe-la-pomme-ne-tombe-jamis-loin-de-l-arbe
- proverbe-les-pomme-ne-tonbe-jamais-loin-de-l-arbre
- proverbe-la-pomme-ne-tombe-jamais-loin-d-arbre

# Type 5 (~ Perrières, FAUX)
5 réponses dont :
- pierriere
- perriere
- derriere
- perrieres
- parrieres

# Type 6 (FAUX)
5 réponses dont :
- rue-de-ma-bite
- rue-de-ma-mere
- rue-dela-mer
- rue-de-ta-mere
- rue-de-la-gare

# Type -1 (réponses à l'ouest)
191 réponses dont :
- boulevard-de-l-ouest
- marthyrs
- proverbe-un-pommier
- sans-deconner-comment-je-peux-deviner-sans-tricher
- rennes
- rue-des-vignerons
- chassenolles
- jupilles
- rue-de-montresor
- rue-de-l-hermitage
- rue-du-marche
- nom-spam
- 43-grande-rue
- hfdhfg
- montresor

## Hélas parmi les réponses à l'ouest il y a des réponses qu'on pourrait considérer comme bonnes :

- rue-de-la-couteauderie-37460-montresor
- sur-la-rue-couteauderie